In [46]:
%matplotlib inline

In [47]:
import hvplot.pandas

In [130]:
import pandas as pd
from fastcore import *
from fastcore.all import * 

In [131]:
base_path = Path('/mnt/c/trial/results')

In [132]:
experiment_paths = base_path.ls()
experiment_paths

(#1) [Path('/mnt/c/trial/results/new-minisat-localrestart')]

In [133]:
def read_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.read().splitlines()
        
    d = {}
    for l in lines:
        value = l.split(':')
        d[value[0]] = value[1]
    
    return d

In [134]:
def read_experiment_results(name):
    exp = base_path / name
    df = pd.DataFrame([read_file(file_name) for file_name in exp.ls()])
    df = df.astype({'nclauses': int, 'num_assignments': int, 'num_backtracks': int, 'num_decisions': int, 'num_learned': int, 
                    'nvars': int, 'runtime': float})
    df['experiment'] = name
    return df

In [135]:
a = experiments[0]
experiment_names = [p.name for p in experiment_paths]
experiment_names

['new-minisat-localrestart']

In [136]:
df = pd.concat([read_experiment_results(name) for name in experiment_names], axis=0)

In [137]:
df.head(3).T

,0,1,2
filename,3col200_5_3.shuffled.cnf,3col200_5_5.shuffled.cnf,3col20_5_10.shuffled.cnf
nclauses,1706,1706,176
num_assignments,3031851,3752229,269
num_backtracks,31668,31011,23
num_decisions,107795,219436,72
num_learned,31668,31011,23
num_restarts,3568,9985,3
nvars,400,400,40
runtime,41.002,70.141,0.0
solver,new_solver,new_solver,new_solver


# Check for errors

In [138]:
df.groupby(['error'])['error'].count()

error
Found conflict at level dl=0            125
Timeout of **1200** seconds exceeded      5
Name: error, dtype: int64

Great! No runtime errors! Only conflicts and timeouts!

# Statistics per experiment

In [144]:
df.groupby(['experiment', 'solver', 'vdh', 'status']).agg({'experiment': 'count', 'runtime': 'mean',
                'num_assignments': 'mean', 'num_learned': 'mean', 'num_backtracks': 'mean', 'nclauses': 'mean'}).\
                    rename({'experiment': 'count'}, axis=1)

count      runtime  \
experiment               solver     vdh     status                       
new-minisat-localrestart new_solver MiniSAT ERROR       5  1200.012200   
                                            SAT       268    19.028612   
                                            UNSAT     125    34.156104   

                                                    num_assignments  \
experiment               solver     vdh     status                    
new-minisat-localrestart new_solver MiniSAT ERROR      2.371692e+07   
                                            SAT        2.150957e+06   
                                            UNSAT      3.132378e+06   

                                                      num_learned  \
experiment               solver     vdh     status                  
new-minisat-localrestart new_solver MiniSAT ERROR   667669.800000   
                                            SAT       5220.902985   
                                            UNSAT     8998.680000   

                                                    num_backtracks  \
experiment               solver     vdh     status                   
new-minisat-localrestart new_solver MiniSAT ERROR    667669.800000   
                                            SAT        5220.902985   
                                            UNSAT      8998.680000   

                                                       nclauses  
experiment               solver     vdh     status               
new-minisat-localrestart new_solver MiniSAT ERROR   4212.200000  
                                            SAT     4796.608209  
                                            UNSAT   4351.208000

In [140]:
df.hvplot.scatter(x='nclauses', y='runtime', by=['status']).options(title='Runtime by #clauses')

:NdOverlay   [status]
   :Scatter   [nclauses]   (runtime)